In [1]:
import pandas as pd
from collections import OrderedDict
import torch
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


Bad key "text.kerning_factor" on line 4 in
C:\Users\MARCELO\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
'''
EXTERNAL KERNEL
'''
google_colab = False
kaggle = False

'''
CUDA
'''
cuda = False

'''
DATA REPRESENTATION

1 => SINGLE READ | 2 => ADD FEATURES | 3 => WINDOW TO FEATURES
'''
DATA_REPRESENTATION = 2

'''
DOWNSAMPLE FACTOR

1 => 10hz *original rate* | 2 => 5Hz | 5 => 2Hz | 10 => 1hz
'''
DOWNSAMPLE_FACTOR = 5

'''
WINDOWS LENGHT

* needs divisor by datapoints target
* considering downsample factor = 5

1 => WINDOW DISABLED | 2 => 1 second | 4 => 2 seconds | 10 => 5 seconds | 20 => 10 seconds | 200 => 100 seconds *full flight*
'''
WINDOW_LENGHT =  1

'''
LIMITADOR

Quantity of samples in the execution of the tests.
'''
LIMITADOR = 20000

'''
LOSS FACTOR [0,1]

Ignores outliers in calculating the stats of losses in regenerated data.
'''
LOSS_FACTOR = .96

'''
TRAIN_SIZE [0,1]

Percentage of samples to be trained
'''
TRAIN_SIZE = 0.8

'''
PATH_DATASET

'''
PATH_DATASET = '../../dataset/original/'

'''
OUTPUT_FILE_NAME

File with output results
'''
OUTPUT_FILE_NAME = 'output-dr_' + str(DATA_REPRESENTATION) + '-ts_' + str(TRAIN_SIZE) + '-lf_' + str.replace(str(LOSS_FACTOR), '.', '') + '-wl_' + str(WINDOW_LENGHT) + '_final.txt'

In [3]:
if google_colab:
    from google.colab import drive

    drive.mount('/content/drive')
    path = '/content/drive/My Drive/ACADÊMICO/MESTRADO/DISSERTAÇÃO/CHAPTERS/5 EXPERIMENTO/dataset/data_representation_1'
    dict_ds_original = {
        'data_ds3_normal_t1_original' : pd.read_csv(path+'/F16_DS3_normal_t1.csv', header=None),
        'data_ds3_normal_t2_original' : pd.read_csv(path+'/F16_DS3_normal_t2.csv', header=None),
        'data_ds3_fault1_original' : pd.read_csv(path+'/F16_DS3_fault1_leakage.csv', header=None),
        'data_ds3_fault2_original' : pd.read_csv(path+'/F16_DS3_fault2_viscousfriction.csv', header=None),
        'data_ds3_fault3_original' : pd.read_csv(path+'/F16_DS3_fault3_compressibility.csv', header=None),
        'data_ds3_fault4_original' : pd.read_csv(path+'/F16_DS3_fault4_fixedposition.csv', header=None),
    }
elif kaggle:
    !conda install -y gdown
    !gdown --id 1G88okIVmdcgLFlmd7rDRhHvHv98yK3UB
    !gdown --id 1fX3utfHMjwKTt7IW4D01bnm-hv88yzrJ
    !gdown --id 1yUG3R5zK2AIxtS9Q4Fk-udkKBZeYShgb
    !gdown --id 1OBRDtuqNEZ-3Z-q0helWh2xGiAxeLACH
    !gdown --id 17oDi60sWYsWHHxzj2aA9m6ARm8zQ81m_
    !gdown --id 1jKEK4s5sYJh8PHtpHeV8ABOsHjuB26RA
else:
    dict_ds_original = {
        'data_ds3_normal_t1_original' : pd.read_csv(PATH_DATASET+'F16_DS3_normal_t1.csv', header=None),
        'data_ds3_normal_t2_original' : pd.read_csv(PATH_DATASET+'F16_DS3_normal_t2.csv', header=None),
        'data_ds3_fault1_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault1_leakage.csv', header=None),
        'data_ds3_fault2_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault2_viscousfriction.csv', header=None),
        'data_ds3_fault3_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault3_compressibility.csv', header=None),
        'data_ds3_fault4_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault4_fixedposition.csv', header=None),
    }

In [4]:
dict_ds = dict_ds_original.copy()

if dict_ds['data_ds3_normal_t1_original'].shape[0] % DOWNSAMPLE_FACTOR != 0 or dict_ds['data_ds3_fault1_original'].shape[0] % DOWNSAMPLE_FACTOR != 0:
    raise Exception('Needs to be ?shape? divisor')

for n, dataset_name in enumerate(dict_ds):
    dataset = dict_ds[dataset_name].to_numpy()

    downsampled = dataset[::DOWNSAMPLE_FACTOR]

    x, y = downsampled.shape

    # resample
    dict_ds[dataset_name] = pd.DataFrame(downsampled.reshape((int(x/WINDOW_LENGHT),y*WINDOW_LENGHT)))


In [5]:
# ADD COLUMNS WITH DIFF PREVIOUS VALUES

if (DATA_REPRESENTATION == 2):
    frame_size = int(1000/DOWNSAMPLE_FACTOR)

    for n, dataset_name in enumerate(dict_ds):
        dataset = dict_ds[dataset_name].to_numpy()

        dimension = dataset.shape[1]
        samples = dataset.shape[0]

        # GENERATE NEW DIMENSIONS
        dataset = np.concatenate((dataset, np.zeros((samples,dimension))), axis=1)

        for f in np.arange(0,int(samples/frame_size)):
            # OBTAIN THE FRAME FLIGHT
            frame = dataset[f*frame_size:(f+1)*frame_size, 0:dimension]

            # CALCULATE DIFFERENCE
            chunk = np.diff(frame, axis=0)

            # DONT CALCULATE THE DIFFERENCE FOR EACH FIRST TIMESTEP
            chunk = np.insert(chunk, 0, frame[0, 0:dimension], axis=0)

            # UPDATE DATASET WITH NEW FRAME INTO NEW DIMENSIONS
            dataset[f*frame_size:(f+1)*frame_size,dimension:dimension*2] = chunk

        dict_ds[dataset_name] = pd.DataFrame(dataset)



In [6]:
ss = StandardScaler()

data_ds3_t1_normal = dict_ds['data_ds3_normal_t1_original']
data_ds3_t2_normal = dict_ds['data_ds3_normal_t2_original']
data_ds3_fault1 = dict_ds['data_ds3_fault1_original']
data_ds3_fault2 = dict_ds['data_ds3_fault2_original']
data_ds3_fault3 = dict_ds['data_ds3_fault3_original']
data_ds3_fault4 = dict_ds['data_ds3_fault4_original']

# fit values
ss.partial_fit(data_ds3_t1_normal)
ss.partial_fit(data_ds3_t2_normal)
ss.partial_fit(data_ds3_fault1)
ss.partial_fit(data_ds3_fault2)
ss.partial_fit(data_ds3_fault3)
ss.partial_fit(data_ds3_fault4)

# transform values
data_ds3_t1_normal = ss.transform(data_ds3_t1_normal)
data_ds3_t2_normal = ss.transform(data_ds3_t2_normal)
data_ds3_fault1 = ss.transform(data_ds3_fault1)
data_ds3_fault2 = ss.transform(data_ds3_fault2)
data_ds3_fault3 = ss.transform(data_ds3_fault3)
data_ds3_fault4 = ss.transform(data_ds3_fault4)

# append normal labels
data_ds3_t1_normal = np.append(data_ds3_t1_normal, np.zeros((data_ds3_t1_normal.shape[0],1)), axis = 1)
data_ds3_t2_normal = np.append(data_ds3_t2_normal, np.zeros((data_ds3_t2_normal.shape[0],1)), axis = 1)

# append fault labels
def generate_fault_label(dataset, fault_label):
    labels = np.array([[fault_label]]*dataset.shape[0])

    return labels

data_ds3_fault1 = np.append(data_ds3_fault1, generate_fault_label(data_ds3_fault1, 1), axis = 1)
data_ds3_fault2 = np.append(data_ds3_fault2, generate_fault_label(data_ds3_fault2, 2), axis = 1)
data_ds3_fault3 = np.append(data_ds3_fault3, generate_fault_label(data_ds3_fault3, 3), axis = 1)
data_ds3_fault4 = np.append(data_ds3_fault4, generate_fault_label(data_ds3_fault4, 4), axis = 1)

In [7]:
dimension = data_ds3_t2_normal.shape[1]-1

# FUNCTIONS AND CLASSES
class Autoencoder(nn.Module):
    def __init__(self, encode_l, decode_l):
        super().__init__()
        self.encoder = nn.Sequential(encode_l)
        self.decoder = nn.Sequential(decode_l)

    def forward(self, x):
        return self.decoder(self.encoder(x))

def run_train(net, train_loader, num_epochs, optimizer, loss_func):
    train_loss = []
    for epoch in range(num_epochs):
        running_loss = 0.0
        losses = []
        for n, (real_samples, _) in enumerate(train_loader):

            net.zero_grad()

            ### forward ###
            if cuda:
                output = net(real_samples.type(torch.FloatTensor).cuda())
                loss = loss_func(output, real_samples.type(torch.FloatTensor).cuda())
            else:
                output = net(real_samples.type(torch.FloatTensor))
                loss = loss_func(output, real_samples.type(torch.FloatTensor))

            running_loss += loss.item()
            losses.append(loss.item)

            ### backward ###
            loss.backward()
            optimizer.step()

        step_loss = running_loss / len(train_loader)
        train_loss.append(step_loss)

        ### log ###
        #print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss))

    return net, output, train_loss, losses

def generate_ground_truth(data_test):
    ground_truth = []

    for j, x in enumerate(data_test):

        if x.shape[0] == dimension: # X_test
            ground_truth.append(0)
        else: # % others: get column label
            if x[-1] != 0:
                ground_truth.append(1)
            else:
                ground_truth.append(0)

    return ground_truth

def generate_losses(data_test, net, loss_function):
    losses = []
    #regenerate_data = np.zeros((1, dimension))
    for j, x in enumerate(data_test):

        if x.shape[0] > dimension:
            x = x[:-1]

        real = x.reshape(1,-1).astype(np.float32)

        #regenerate_data[j] = regenerate.cpu().detach().numpy()

        if cuda:
            regenerate = net(torch.from_numpy(real).cuda())
            loss_ae = loss_function(regenerate, torch.from_numpy(real).cuda()).item()
        else:
            regenerate = net(torch.from_numpy(real))
            loss_ae = loss_function(regenerate, torch.from_numpy(real)).item()

        losses.append(loss_ae)

    return losses

def generate_y_hat(losses, loss_threshold):
    y_hat = []

    for l in losses:
        if l < loss_threshold:
            y_hat.append(0)
        else:
            y_hat.append(1)

    return y_hat

def tester(data_test, net, loss_function, loss_threshold = 1):

    ground_truth = []
    losses = []

    for n, dataset_name in enumerate(data_test):
        dataset = data_test[dataset_name]

        ground_truth = ground_truth + generate_ground_truth(dataset)

        losses = losses + generate_losses(dataset, net, loss_function)

    y_hat = generate_y_hat(losses, loss_threshold)

    return confusion_matrix(ground_truth, y_hat, normalize='true'), losses, ground_truth, y_hat

def generate_encode_decode_layers(layers, output_layer):
    od_encode = []
    od_decode = []
    # encode
    for _, layer in enumerate(layers):
        n = _ + 1

        if (len(layers) == n):
            break

        od_encode.append(('l'+str((len(od_encode)+1)), nn.Linear(layers[_],layers[n])))

        if (len(layers) != n+1):
            od_encode.append(('l'+str((len(od_encode)+1)), nn.ReLU()))

    # decode
    layers.reverse()
    for _, layer in enumerate(layers):
        n = _ + 1

        if (len(layers) == n):
            break

        od_decode.append(('l'+str((len(od_decode)+1)), nn.Linear(layers[_],layers[n])))

        if (len(layers) != n+1):
            od_decode.append(('l'+str((len(od_decode)+1)), nn.ReLU()))
        else:
            od_decode.append(('l'+str((len(od_decode)+1)), output_layer))

    return OrderedDict(od_encode), OrderedDict(od_decode)

def train(layers, last_layer, lr, epochs, batch_size, X_train, optim, loss_fnc, net = []):
    encode_l, decode_l = generate_encode_decode_layers(layers, last_layer)

    if (net == []):
        net = Autoencoder(encode_l, decode_l)

    if cuda:
        net.cuda()

    if (optim == 'ADAM'):
        optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=1e-5)
    elif(optim == 'SGD'):
        optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    elif(optim == 'RMSprop'):
        optimizer = torch.optim.RMSprop(net.parameters(), lr=lr)

    torch.manual_seed(111)

    # sets
    train_set = [
        (X_train, X_train) for i in range(len(X_train))
    ]

    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, shuffle=False
    )

    # train
    net, output, loss, losses = run_train(net, train_loader, epochs, optimizer, loss_fnc)

    return net, output, loss, losses, loss_fnc

def test(X_train, X_test, net, loss_function, phi = 0):

    faults = {'F1': data_ds3_fault1[:LIMITADOR],
              'F2': data_ds3_fault2[:LIMITADOR],
              'F3': data_ds3_fault3[:LIMITADOR],
              'F4': data_ds3_fault4[:LIMITADOR]}

    # CONFUSION MATRIX FOR X_TRAIN
    losses = generate_losses(X_train, net, loss_function)
    np_losses = np.sort(np.array(losses))
    losses = np_losses[:int(len(np_losses)*LOSS_FACTOR)]
    phi_test = np.mean(losses, dtype=np.float64) + np.std(losses, ddof=1, dtype=np.float64)

    if phi == 0:
        phi = phi_test

    print("phi_full: ", phi_test, file=log)
    print("phi_step: ", phi, file=log)

    # CONFUSION MATRIX FOR NORMAL_2
    losses = generate_losses(data_ds3_t2_normal[:LIMITADOR], net, loss_function)
    y_hat_n2 = generate_y_hat(losses, phi)
    print('Normal 2 hit :', len([0 for t in y_hat_n2 if t == 0])/len(y_hat_n2))

    for n, fault_name in enumerate(faults):
        losses = generate_losses(faults[fault_name][:LIMITADOR], net, loss_function)
        y_hat = generate_y_hat(losses, phi)
        print(fault_name + ' hit :', len([1 for t in y_hat if t == 1])/len(generate_y_hat(losses, phi)))

    return y_hat_n2, phi

class Incremental_Stats():

    def __init__(self):
        self.count = 0
        self.mu = 0.0
        self.var = 0.0

    def add(self, x):
        if self.count == 0:
            new_mu = x
            new_var = 0
        else:
            new_mu = (self.count * self.mu + x) / (self.count + 1)
            new_var = ((self.count + 1) / self.count) * (((self.count * self.var) / (self.count + 1)) + (((x - new_mu)**2) / self.count))

        self.count = self.count+1
        self.mu = new_mu
        self.var = new_var

        return self.mu, self.var, self.count

    def get_mean(self):
        return self.mu

    def get_variance(self):
        return self.var

    def get_std(self):
        return self.var**0.5

    def get_n(self):
        return self.count

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data_ds3_t1_normal[:, :-1], data_ds3_t1_normal[:, -1], test_size=1-TRAIN_SIZE, random_state=42)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

architecture = [54, 32, 9]
last_layer = nn.Tanh()
batch_size = 32
optim = 'RMSprop'
loss_fnc = nn.L1Loss()

EPOCHS = 1
block_size = 1000
log = None

block_train = int(X_train.shape[0] / block_size)
net = []
agg_loss = []
count = 0

print('************ HYPERPARAMETERS ************', file=log)
print('LIMITER     :', LIMITADOR, file=log)
print('LOSS FACTOR :', LOSS_FACTOR, file=log)
print(architecture, file=log)
print(last_layer, file=log)
print(batch_size, file=log)
print(optim, file=log)
print(loss_fnc, file=log)

values = np.zeros((block_train, 36))
columns = ['mu_full', 'mu_step', 'mu_agg', 'mu_online', 'std_full',
           'std_step', 'std_agg', 'std_online', 'phi_full', 'phi_step', 'phi_agg', 'phi_online',
           'x_test_hit_full', 'normal_2_hit_full', 'f1_hit_full', 'f2_hit_full', 'f3_hit_full', 'f4_hit_full',
           'x_test_hit_step', 'normal_2_hit_step', 'f1_hit_step', 'f2_hit_step', 'f3_hit_step', 'f4_hit_step',
           'x_test_hit_agg', 'normal_2_hit_agg', 'f1_hit_agg', 'f2_hit_agg', 'f3_hit_agg', 'f4_hit_agg',
           'x_test_hit_online', 'normal_2_hit_online', 'f1_hit_online', 'f2_hit_online', 'f3_hit_online', 'f4_hit_online'] #noorder


columns_order = ['mu_full', 'mu_step', 'mu_agg', 'mu_online', 'std_full',
                 'std_step', 'std_agg', 'std_online', 'phi_full', 'phi_step', 'phi_agg',
                 'phi_online', 'x_test_hit_full', 'x_test_hit_step', 'x_test_hit_agg', 'x_test_hit_online',
                 'normal_2_hit_full', 'normal_2_hit_step', 'normal_2_hit_agg', 'normal_2_hit_online',
                 'f1_hit_full', 'f2_hit_full', 'f3_hit_full', 'f4_hit_full', 'f1_hit_step', 'f2_hit_step',
                 'f3_hit_step', 'f4_hit_step', 'f1_hit_agg', 'f2_hit_agg', 'f3_hit_agg', 'f4_hit_agg',
                 'f1_hit_online', 'f2_hit_online', 'f3_hit_online', 'f4_hit_online']

stats = Incremental_Stats()

for n in np.arange(0, block_train):

    count = count+1
    print('.', end='')

    if block_size*(n+1) < X_train.shape[0]:
        data = X_train[block_size * n : block_size*(n+1)]
    else:
        data = X_train[block_size * n:]

    net, output, loss, losses, loss_function = train(layers=architecture.copy(), last_layer=last_layer, lr=1e-3, epochs=EPOCHS, batch_size=batch_size, X_train=data, optim=optim, loss_fnc=loss_fnc, net=net)

    losses_geral_step = generate_losses(data_test=X_train, net=net, loss_function=loss_fnc)
    np_losses = np.sort(np.array(losses_geral_step))
    losses_geral_step = np_losses[:int(len(np_losses)*LOSS_FACTOR)]
    values[n][0] = np.mean(losses_geral_step, dtype=np.float64)
    values[n][4] = np.std(losses_geral_step, ddof=1, dtype=np.float64)
    phi_full = np.mean(losses_geral_step, dtype=np.float64) + np.std(losses_geral_step, ddof=1, dtype=np.float64)

    losses_batch_step_src = generate_losses(data_test=data, net=net, loss_function=loss_fnc)
    np_losses = np.sort(np.array(losses_batch_step_src))
    losses_batch_step = np_losses[:int(len(np_losses)*LOSS_FACTOR)]
    values[n][1] = np.mean(losses_batch_step, dtype=np.float64)
    values[n][5] = np.std(losses_batch_step, ddof=1, dtype=np.float64)
    phi_batch_step = np.mean(losses_batch_step, dtype=np.float64) + np.std(losses_batch_step, ddof=1, dtype=np.float64)

    agg_loss = np.concatenate((agg_loss, losses_batch_step_src), axis = 0)
    np_losses = np.sort(np.array(agg_loss))
    agg_loss = np_losses[:int(len(np_losses)*LOSS_FACTOR)]
    values[n][2] = np.mean(agg_loss, dtype=np.float64)
    values[n][6] = np.std(agg_loss, ddof=1, dtype=np.float64)
    phi_aggregate = np.mean(agg_loss, dtype=np.float64) + np.std(agg_loss, ddof=1, dtype=np.float64)

    for loss_step in losses_batch_step:
        stats.add(loss_step)

    values[n][3] = stats.get_mean()
    values[n][7] = stats.get_std()
    phi_online = stats.get_mean() + stats.get_std()

    values[n][8] = phi_full
    values[n][9] = phi_batch_step
    values[n][10] = phi_aggregate
    values[n][11] = phi_online

    faults = {'F1': data_ds3_fault1[:LIMITADOR],
              'F2': data_ds3_fault2[:LIMITADOR],
              'F3': data_ds3_fault3[:LIMITADOR],
              'F4': data_ds3_fault4[:LIMITADOR]}

    position = 0
    for p, phi in enumerate([phi_full, phi_batch_step, phi_aggregate, phi_online]):
        # FOR X_TEST
        losses = generate_losses(X_test[:LIMITADOR], net, loss_fnc)
        y_hat_n2 = generate_y_hat(losses, phi)
        values[n][12+position] = len([0 for t in y_hat_n2 if t == 0])/len(y_hat_n2)
        position = position + 1

        # FOR NORMAL_2
        losses = generate_losses(data_ds3_t2_normal[:LIMITADOR], net, loss_fnc)
        y_hat_n2 = generate_y_hat(losses, phi)
        values[n][12+position] = len([0 for t in y_hat_n2 if t == 0])/len(y_hat_n2)
        position = position + 1

        for c, fault_name in enumerate(faults):
            losses = generate_losses(faults[fault_name][:LIMITADOR], net, loss_fnc)
            y_hat = generate_y_hat(losses, phi)
            values[n][12+position] = len([1 for t in y_hat if t == 1])/len(y_hat)
            position = position + 1

values_pd = pd.DataFrame(values, columns=columns)

values_pd[columns_order].transpose()

************ HYPERPARAMETERS ************
LIMITER     : 20000
LOSS FACTOR : 0.96
[54, 32, 9]
Tanh()
32
RMSprop
L1Loss()
......................................................................................................................................

In [ ]:
with open(OUTPUT_FILE_NAME,'a') as f:
    f.write('LOSS FACTOR: ' + str(LOSS_FACTOR) + '\n')
    f.write('ARC: ' + str(architecture) + '\n')
    f.write('LAST LAYER: ' + str(last_layer) + '\n')
    f.write('BATCH: ' + str(batch_size) + '\n')
    f.write('OPTIM: ' + str(optim) + '\n')
    f.write('LOSS FNC: ' + str(loss_fnc) + '\n')
    np.savetxt(f, np.array(columns_order).reshape(1,-1), delimiter=';', fmt='%s')
    np.savetxt(f, values_pd[columns_order].values, delimiter=';', fmt='%f')